In [1]:
%%writefile mapper.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
f = open("stop_words_en.txt", "r") 
stop_words=f.read().split("\n")
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    uniq_words=Counter([ word.lower() for word in words if word.lower() not in stop_words ])
    total=float(sum(uniq_words.values()))
    for word in uniq_words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%f" %(word+":"+"0",1)            
        print "%s\t%f" %(word+":"+article_id,uniq_words[word]/total)
        #print(word)
            

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0.0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = float(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%f" % (current_key, word_sum)
        word_sum = 0.0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%f" % (current_key, word_sum)

Overwriting reducer.py


In [3]:
%%writefile mapper_1.py

import sys
import re
for line in sys.stdin:
    try:
        key,count = line.strip().split('\t', 1)
           
    except ValueError as e:
        continue
    if key=="labor:12" or key=="labor:0":
        print "%s\t%s" % (key,count)


Overwriting mapper_1.py


In [4]:
%%writefile tf_df.py
import sys
from math import log
#print (sys.argv[1],sys.argv[2])
idf_inv=log(1+float(sys.argv[2]))
tf=float(sys.argv[1])
print(tf/idf_inv)

Overwriting tf_df.py


In [5]:
%%bash

OUT_DIR="wordcount_result_1519019602556817"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null 2>/dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2>output.log

OUT_DIR_1="wordcount_result_1519019602566817"
NUM_REDUCERS=1
hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null 2>/dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_1.py \
    -mapper "python mapper_1.py" \
    -input ${OUT_DIR} \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -output ${OUT_DIR_1} > /dev/null 2>>output.log

hadoop dfs -cat wordcount_result_1519019602566817/part* >result.txt 2>/dev/null
t=$(cat result.txt|grep "labor:0"|cut -d":" -f2|cut -f2)
s=$(cat result.txt|grep "labor:12"|cut -d":" -f2|cut -f2)
python tf_df.py $s $t
cat output.log >&2

0.0003505572802883137


18/02/19 07:03:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/19 07:03:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/19 07:03:54 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/19 07:03:54 INFO mapreduce.JobSubmitter: number of splits:2
18/02/19 07:03:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1518945024862_0013
18/02/19 07:03:54 INFO impl.YarnClientImpl: Submitted application application_1518945024862_0013
18/02/19 07:03:54 INFO mapreduce.Job: The url to track the job: http://21ffcf20c7bd:8088/proxy/application_1518945024862_0013/
18/02/19 07:03:54 INFO mapreduce.Job: Running job: job_1518945024862_0013
18/02/19 07:04:01 INFO mapreduce.Job: Job job_1518945024862_0013 running in uber mode : false
18/02/19 07:04:01 INFO mapreduce.Job:  map 0% reduce 0%
18/02/19 07:04:16 INFO mapreduce.Job:  map 6% reduce 0%
18/02/19 07:04:22 INFO mapreduce.Job:  map 8% reduce 0%
18/02/19 07:04:28 INFO ma